In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
# import set

In [2]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()

value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬆️⬆️⬅️⬇️🔄
➡️⏫️⏪⬆️➡️
⬇️➡️⏫️➡️⬇️
⬇️🔄✅⏬⬆️
🔄⏫️🔄⬅️⬇️


In [4]:
# Every-visited  ，下一个cell是first-visited
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.show()
gridworld.showPolicy(policy)
print("random policy")


trajectorySteps = 100
qtable = np.zeros((rows*columns,5))    #生成Qtable，也就是action-value-table
qtable_pre = qtable.copy()+1
while(np.sum((qtable_pre-qtable)**2)>0.001):
    # print(np.sum((qtable_pre-qtable)**2))
    qtable_pre = qtable.copy()
    #通过采样获得action-value的值
    for i in range(rows * columns):    #循环每个state
        for j in range(5):             #循环每个action
            qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)] 
            qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            _, _, score, _, _ = Trajectory[trajectorySteps]
            for k in range(trajectorySteps-1,-1,-1):
                tmpstate, tmpaction, tmpscore,_,_  = Trajectory[k]
                score = score*gamma + tmpscore  #细节从后往前优化算法
                qtable_rewards[tmpstate][tmpaction] += score
                qtable_nums[tmpstate][tmpaction] += 1
                qtable[tmpstate][tmpaction] = qtable_rewards[tmpstate][tmpaction] / qtable_nums[tmpstate][tmpaction]
                #every visit
            # qtable[i,j] = score

    # print("qtable[0]:", qtable[0])
    # print("qtable[1]:", qtable[1])
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    
    gridworld.showPolicy( policy)
    print("action value's mean",qtable.mean())



    

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬆️⬅️⬇️⬅️⬅️
🔄⏫️🔄🔄⬆️
🔄⬇️⏩️⬅️🔄
⬅️⏩️✅🔄⬅️
⬅️⏪➡️🔄⬇️
random policy
⬇️🔄⬅️⬇️⬇️
⬇️⏪⏩️🔄⬇️
⬆️⬅️⏬⬆️🔄
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬅️
action value's mean -31.66122803357247
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.29277346688155703
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.5552036922867665
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.791389917691976
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.8998281594850077
➡️➡️➡️⬇️⬇️
⬆️⏫️⏩️⬇️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.9688942318450082
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.968894231845008


In [11]:
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.show()
gridworld.showPolicy(policy)
print("random policy")


trajectorySteps = 100
qtable = np.zeros((rows*columns,5))    #生成Qtable，也就是action-value-table
qtable_pre = qtable.copy()+1
while(np.sum((qtable_pre-qtable)**2)>0.001):
    print(np.sum((qtable_pre-qtable)**2))
    qtable_pre = qtable.copy()
    #通过采样获得action-value的值
    for i in range(rows * columns):    #循环每个state
        for j in range(5):             #循环每个action
            # qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)] 
            # qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            _, _, score, _, _ = Trajectory[trajectorySteps]
            for k in range(trajectorySteps-1,-1,-1):
                tmpstate, tmpaction, tmpscore,_ ,_  = Trajectory[k]
                score = score*gamma + tmpscore  #细节从后往前优化算法
                # qtable_rewards[tmpstate][tmpaction] += score
                # qtable_nums[tmpstate][tmpaction] += 1
                qtable[tmpstate][tmpaction] = score  #first visit
                
            #qtable[i,j] = score

    print("qtable[0]:", qtable[0])
    print("qtable[1]:", qtable[1])
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    
    gridworld.showPolicy(policy)
    # print(np.sum((qtable_pre-qtable)**2))



    

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬆️⬇️⬆️🔄🔄
🔄⏩️⏫️⬇️⬇️
➡️⬇️⏬➡️⬇️
⬆️⏪✅⏩️➡️
🔄⏩️⬅️⬅️🔄
random policy
125.0
qtable[0]: [ -9.99973439 -23.66076095   0.          -9.99976095  -8.99976095]
qtable[1]: [-24.66076095  -8.99976095 -26.28973439  -8.99976095 -23.66076095]
⬇️➡️➡️➡️⬅️
🔄⏪⏬⬆️⬆️
⬆️➡️⏬⬅️⬆️
⬇️⏩️✅⏪⬇️
🔄⏪⬆️➡️🔄
58724.65101582276
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬆️
54289.483682466176
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬅️
668.2177293353504
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
265.70307335590013
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
215.21792104600723
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬇️
⬆️⬅️⏬